# Principal Component Analysis (PCA)

PCA identifies the axes that correspond to the greatest variation in a dataset. Usually, most of the variation in a dataset can be summarized by a few principal components. Therefore, the structure of a dataset can be represented using only several principal components.

## 1. Introduction: Why reduce dimensions? The curse of dimensionality

Consider a circle (n-sphere) inscribed in a square (n-cube), and view how much of the n-cube is covered by the n-sphere as the number of dimensions increases:

<img src="http://datasets.genepattern.org/images/sphere-in-cube.png">

As the number of dimensions increases, the volume of space covered by a unit sphere becomes proportionally smaller.

- The volume of an $n$-ball, where $n$ is the dimension and $R$ is the radius, is $V_{n}(R) = \frac{\pi^{\frac{n}{2}}}{\Gamma({\frac{n}{2} + 1})} R^{n}$

- The volume of an $n$-cube of side length $d$ is $d^{n}$.

- We will calculate the ratio of an inscribed $n$-sphere in an $n$-cube as a function of the dimension $n$:

### 1.1. Perform setup steps

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Execute the cells below.
</div>

In [73]:
# Import required libraries for computing and plotting:
import math
from math import gamma
import matplotlib.pyplot as plt

# Enable embedding matplotlib graphs in the notebook:
%%matplotlib inline

In [82]:
# Compute the sphere volume, cube volume, and ratio for the number of dimensions selected:
radius = 1  
num_dimensions = 10
dimensions = range(1,num_dimensions+1)

volume_sphere = [0] * num_dimensions
volume_cube = [0] * num_dimensions
ratio = [0] * num_dimensions

for idx, n in enumerate(dimensions):
    volume_cube[idx] = ((radius*2)**n)
    volume_sphere[idx] = ((math.pi**(n/2.0))*(radius**n))/gamma(n/2.0+1)
    ratio[idx] = volume_sphere[idx] / volume_cube[idx]

### 1.2. Create plots using matplotlib

In [ ]:
# Plot the resulting data:

fig = plt.figure(figsize=(10,10))
plt.clf()

x_vals = dimensions

ax = fig.add_subplot(3,1,1)
bars = plt.bar(x_vals, volume_sphere, 0.8)
plt.xticks(dimensions)
plt.xlabel("Volume of unit sphere of dimension n")

ax = fig.add_subplot(3,1,2)
bars = plt.bar(x_vals, volume_cube, 0.8)
plt.xticks(dimensions)
plt.xlabel("Volume of cube of s=2, dimension n")

ax = fig.add_subplot(3,1,3)
bars = plt.bar(x_vals, ratio, 0.8)
plt.xticks(dimensions)
plt.xlabel("Ratio of sphere to enclosing cube")

plt.show()

### 1.3. Discussion

From the plots above, you can see that as the dimension increases, the amount of empty space quickly dominates the interior space of a unit sphere. This means that, the more dimensions there are in your data, the more difficult it is for your samples to cover the space of your dataset.

This challenge is illustrated in the graphic below:

<img src="http://datasets.genepattern.org/images/samples-per-dimension.png" style="height: 500px;"/>
<p style="text-align: center;font-weight: bold;"> Number of samples required to cover 60% of sample space</p>

## 2. Principal Component Analysis
Defines a set of “principal components”
- First Principal Component: direction of greatest variability in data
- Second Principal Component: perpendicular to 1st; greatest variability of remaining data
- so on until all dimensions are accounted for

The top principal components represent the data
- The number of top PCs is much smaller than the total number of dimensions
- The method represents every data point in the dimensions of the principal components

<img src="http://datasets.genepattern.org/images/pca-demo.png">

## 2. Run PCA in GenePattern

### 2.1 Sign in to GenePattern

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<ul>
	<li>If you haven&#39;t yet logged in, enter your credentials into the cell below and click Login.</li>
    <li>In the server dropdown, make sure you select <b>GenePattern Cloud</b>.</li>
</ul></div>

In [13]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

### 2.2. Run PCA in GenePattern to compute the principal components of the dataset

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<ul>
	<li>Click and drag the following breast cancer dataset link to the <strong>input filename</strong> parameter below: <a href="https://datasets.genepattern.org/data/ccmi_tutorial/2017-12-15/BRCA_HUGO_symbols.preprocessed.gct" target="_blank">BRCA_HUGO_symbols.preprocessed.gct</a></li>
	<li>Notice we are clustering by <strong>columns</strong>, which correspond to samples. This means we will be observing which samples cluster with one another.</li>
	<li>Click <strong>Run</strong></li>
</ul>
</div>

In [14]:
pca_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00017')
pca_job_spec = pca_task.make_job_spec()
pca_job_spec.set_parameter("input.filename", "")
pca_job_spec.set_parameter("cluster.by", "3")
pca_job_spec.set_parameter("output.file", "<input.filename_basename>")
genepattern.display(pca_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00017')

When the job completes, you will see a new panel above with the title **Job # XXXXX**, where the XXXXX corresponds to the GenePattern job ID of your PCA analysis. You will also see 4 result files:

Filename | Description
:------------ | :-------------
`BRCA_HUGO_symbols.preprocessed_s.odf` | the **s matrix (eigenvectors)**
`BRCA_HUGO_symbols.preprocessed_t.odf` | the **t matrix (transformed original dataset)**
`BRCA_HUGO_symbols.preprocessed_u.odf` | the **u matrix (eigenvalues)**
`gp_execution_log.txt` | the execution log - a record of the analysis run

## 3. Visualize the PCA results
To visualize the results of the PCA analysis, we will read the **s matrix** and **u matrix files** into Python array structures, and create graphs based on the arrays. We do not need the **t matrix** for this analysis.

### 3.1. Load required Python libraries

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Execute the cell below.
</div>

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import genepattern
from matplotlib.ticker import FuncFormatter

### 3.2. Read file results into Python variables

The GenePattern results are files on the GenePattern server. To read them into Python arrays, we will use the "`Send to code`" functionality in GenePattern Notebook.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<div class="alert alert-info">
<ol>
<li>In the cell below, click the down arrow on the **gp s matrix file** parameter and select BRCA_HUGO_symbols.preprocessed_s.odf</li>
<li>In the cell below, click the down arrow on the **gp u matrix file** parameter and select BRCA_HUGO_symbols.preprocessed_u.odf</li>
	<li>Execute the cell below by clicking <strong>Run</strong>.</li>
</ol>
</div>
</div>

In [15]:
@genepattern.build_ui(name="Convert GenePattern ODF Matrix model result files to numpy arrays", 
                    description="Take as input the S and U matrices resulting from a GenePattern PCA job " +
                                "and convert then to numpy arrays", 
                    parameters={
                                "gp_s_matrix_file":{"type": "file", "kinds": ["odf"]},
                                "gp_u_matrix_file":{"type": "file", "kinds": ["odf"]},
                                "output_var": {"hide": False}
                                })

def pca_results_to_arrays(gp_s_matrix_file, gp_u_matrix_file):
    s_matrix_array = gp_matrix_odf_to_nparray(gp_s_matrix_file)
    print("Converted S matrix array.")
    u_matrix_array = gp_matrix_odf_to_nparray(gp_u_matrix_file)
    print("Converted U matrix array.")
    print("Conversion complete.")
    return s_matrix_array, u_matrix_array
    
def gp_matrix_odf_to_nparray(gp_file):
    
    # Get a reference to your session with the GP server
    gpserver = genepattern.session.get(0)

    # Create the GPFile object (using the right credentials)
    fh = gp.GPFile(gpserver, gp_file)
    
    matrix_bytes = fh.read()

    # Remove header lines
    matrix_string = re.sub(".*\n", '', matrix_bytes, count=5, flags=0)
    matrix_string = re.sub("\t\n", '\n', matrix_string, count=0, flags=0)

    # The final split leaves an extra line, which must be removed
    matrix_list = matrix_string.split('\n')
    matrix_list.pop(len(matrix_list)-1)

    matrix_2dlist = [row.split('\t') for row in matrix_list]

    # Populate the new array with contents of the list:
    matrix_array = np.empty(shape=(len(matrix_2dlist),len(matrix_2dlist[0])))
 
    for r in range(len(matrix_2dlist)):
        for c in range(len(matrix_2dlist[0])):
            matrix_array[r][c] = matrix_2dlist[r][c]
            
    return(matrix_array)



UIBuilder(description='Take as input the S and U matrices resulting from a GenePattern PCA job and convert the…

### 3.3. Read phenotype assignments to each sample

We will next read the file that contains the phenotype assignments (e.g., tumor, normal, etc.) for the samples in our dataset. These are in the [CLS](http://software.broadinstitute.org/cancer/software/genepattern/file-formats-guide#CLS) file format.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<ul>
	<li>Click and drag the file containing the phenotype descriptions to the <strong>cls file url</strong> parameter below: <a href="https://datasets.genepattern.org/data/ccmi_tutorial/2017-12-15/BRCA_HUGO_symbols.preprocessed.cls" target="_blank">BRCA_HUGO_symbols.preprocessed.cls</a></li>
	<li>Click <strong>Run</strong></li>
</ul>
</div>

In [16]:
@genepattern.build_ui(name="Read a phenotype assignment file (cls format) from a url and return its data", 
                      description="Take as input the url to a cls file and return the data it contains: " +
                                    "number of samples, number of classes, class names, class assignments", 
                      parameters={
                          "output_var": {"hide": False}
                    })

def read_phenotype_assignments(cls_file_url):
    cls_file = urllib.request.urlopen(cls_file_url)
    l1 = cls_file.readline()
    (num_samples, num_classes, one) = [int(i) for i in l1.split()]

    l2 = cls_file.readline()
    class_names = l2.split()
    class_names.pop(0)

    l3 = cls_file.readline()
    class_assignments = [int(i) for i in l3.split()]
    
    print("Read class assignments", class_names)
    
    return (num_samples, num_classes, class_names, class_assignments)


UIBuilder(description='Take as input the url to a cls file and return the data it contains: number of samples,…

### 3.4. Set up Python variables for plotting

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Execute the cell below.
</div>

In [55]:
# Extract the s and u matrices from the results
(s_matrix, u_matrix) = result_matrices

# The principal components are the transpose of the u matrix:
pc = u_matrix.transpose()

# Convert eigenvectors from an array to a list
# The eigenvector matrix only has entries on the diagonal. Extract these into a list to facilitate processing:
evectors = [s_matrix[x][x] for x in range(len(s_matrix))]

# Compute percentage contribution of each eigenvector
ev_total = sum(evectors)
ev_percents = evectors/ev_total

# The `class_data` variable contains the class information - parse it out into variables:
(num_samples, num_classes, class_names, class_assignments) = class_data

# Create color map for up to 6 classes:
colormap = ["#ff0000","#0000ff", "#00ff00", "#00ffff", "#ff00ff", "#ffff00"]
colors = [colormap[class_assignments[i]] for i in range(len(class_assignments))]

###  3.5. Display scatter plot of first 3 principal components

This step will use a different plotting library called [Plotly](https://plot.ly). Plotly provides graphing libraries that are especially useful when you want to provide interactive plots. In this case, you can use the mouse click, drag, and zoom to change the rotation or magnification of the 3D view.

#### 3.5.1 Perform setup steps

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Execute the cells below.
</div>

In [56]:
# Import required Python libraries
import plotly
import plotly.graph_objs as go

# Allow embedding of plots in the output cell of the notebook
plotly.offline.init_notebook_mode() 

#### 3.5.2. Create 3D graph of first 3 principal components using Plotly

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<ol>
<li>Execute the cell below.</li>
<li>You will see a 3D scatter plot of the first 3 principal components of the dataset.</li>
<li>Explore the dataset:<p>
<ul>
<li>Click and drag to rotate left, right, up, or down</li>
<li>Scroll to zoom in and out</li>
<li>Hover over a point to see its data value.</li></li>
</ul>
</ol>
</div>

In [ ]:
data=[]
traces=[]

for j in range(num_classes):
    traces.append(go.Scatter3d(
    x=[i for indx,i in enumerate(pc[0]) if class_assignments[indx] == j],
    y=[i for indx,i in enumerate(pc[1]) if class_assignments[indx] == j],
    z=[i for indx,i in enumerate(pc[2]) if class_assignments[indx] == j],
    mode='markers',
    name=str(class_names[j].decode("utf-8")),
    marker=dict(
        size='10',
        color=colormap[j]
        )
    ))
    data.append(traces[j])
    
layout=go.Layout(dict(height=1000, width=1000, 
            title='TCGA Breast Cancer vs Normal PCA',
            scene=dict(xaxis=dict(title="PC 1",visible=True),
            yaxis=dict(title="PC 2",visible=True),
            zaxis=dict(title="PC 3",visible=True))
            )
)

fig=dict(data=traces, layout=layout)
plotly.offline.iplot(fig)

### 3.6. Display percentage of variance explained for each principal component

Variance explained is the proportion of dispersion of the dataset that is covered by each principal component. It provides an idea of how many principal components are required to represent the majority of the dataset.

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Execute the cell below.
</div>

In [ ]:
plt.figure(figsize=(15,15))

plt.clf()

def percents(x, pos):
    'The two args are the value and tick position'
    return '%1.1f%%' % (x * 100)

formatter = FuncFormatter(percents)

plt.title("Variance Explained Per Principal Component")
x_vals = [i for i in range(num_samples)]
bars = plt.bar(x_vals, ev_percents, 0.8)

plt.xlabel("Principal Component")
plt.ylabel("Variance Explained")
plt.show()

## 4. Extra credit

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
<ol>
<li>For convenience, move the cell below using the '&#8593;' button in the menu bar so that it is right above the GenePattern PCA cell.</li>
<li>Re-open the PCA cell by clicking the '+' button in the upper right-hand corner.</li>
<li>Re-run the workflow with the new files, starting with the PCA analysis.</li>
</ol>
</div>

## Extra credit analysis
- Perform the PCA analysis on the following files, which consist of 38 samples comprising two leukemia subtypes, ALL and AML. The rightmost column indicates where you should drag their urls.

Filename | Description | Send to this notebook parameter
:------------ | :------------- | :-------------
[all_aml_preprocessed.gct](https://datasets.genepattern.org/data/all_aml/all_aml_test.preprocessed.gct) | Gene expression file | PCA analysis cell **input filename** parameter
[all_aml_train.cls](https://datasets.genepattern.org/data/all_aml/all_aml_train.cls) | Phenotype assignments file | "Read a phenotype" analysis cell **cls file url** parameter